In [1]:
# Import modules
import pandas as pd
import h2o

In [2]:
# Start a local H2O cluster
h2o.init(nthreads = -1)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_72"; Java(TM) SE Runtime Environment (build 1.8.0_72-b15); Java HotSpot(TM) 64-Bit Server VM (build 25.72-b15, mixed mode)
  Starting server from /Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/4z/p7yt7_4n4fj1jlyq6g4qhfbw0000gn/T/tmpho0r2p20
  JVM stdout: /var/folders/4z/p7yt7_4n4fj1jlyq6g4qhfbw0000gn/T/tmpho0r2p20/h2o_jofaichow_started_from_python.out
  JVM stderr: /var/folders/4z/p7yt7_4n4fj1jlyq6g4qhfbw0000gn/T/tmpho0r2p20/h2o_jofaichow_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.14.0.6
H2O cluster version age:,4 months !!!
H2O cluster name:,H2O_from_python_jofaichow_6rld0a
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
# Import data as H2O Data Frame
h_train = h2o.import_file("./data/train.csv")
h_test = h2o.import_file("./data/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# Define features (or predictors) - notes: exclude 'Id' and target 'SalePrice' 
features = h_train.col_names
features.remove('Id')
features.remove('SalePrice')
features

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'Enc

In [5]:
# Build a Distributed Random Forest (DRF) model with default settings

# Import the function for DRF
from h2o.estimators.random_forest import H2ORandomForestEstimator

# Set up DRF for regression with 5-fold cross-validation
# Add a seed for reproducibility
drf_baseline = H2ORandomForestEstimator(model_id = 'drf_default', 
                                        nfolds = 5, 
                                        seed = 1234)

# Use .train() to build the model
drf_baseline.train(x = features, 
                   y = 'SalePrice', 
                   training_frame = h_train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [6]:
# Model Summary
drf_baseline

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  drf_default


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 837934511.4267565
RMSE: 28947.098497548188
MAE: 16824.085178911897
RMSLE: 0.13965617775881853
Mean Residual Deviance: 837934511.4267565

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 814775750.9682842
RMSE: 28544.277026547446
MAE: 17009.127417487158
RMSLE: 0.14037094407475256
Mean Residual Deviance: 814775750.9682842
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,17035.605,1075.0117,15295.1875000,15951.877,16214.911,18994.043,18722.012
mean_residual_deviance,816360580.0000000,150756736.0000000,632738750.0000000,747103230.0000000,616277820.0000000,1194732930.0000000,890950080.0000000
mse,816360580.0000000,150756736.0000000,632738750.0000000,747103230.0000000,616277820.0000000,1194732930.0000000,890950080.0000000
r2,0.8717843,0.0165829,0.8779815,0.8840764,0.8959165,0.8273461,0.8736007
residual_deviance,816360580.0000000,150756736.0000000,632738750.0000000,747103230.0000000,616277820.0000000,1194732930.0000000,890950080.0000000
rmse,28345.225,2540.5469,25154.299,27333.19,24824.943,34564.91,29848.787
rmsle,0.1396314,0.0126747,0.1264608,0.1210312,0.1289151,0.1549750,0.1667752


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-02-10 07:54:20,14.149 sec,0.0,nan,nan,nan
,2018-02-10 07:54:20,14.255 sec,1.0,39921.0692468,27332.7191651,1593691769.8083491
,2018-02-10 07:54:20,14.310 sec,2.0,42055.6425565,27512.5716783,1768677070.8365386
,2018-02-10 07:54:20,14.363 sec,3.0,41223.2425973,26505.5004372,1699355730.2327383
,2018-02-10 07:54:20,14.416 sec,4.0,39749.5426415,25397.6899911,1580026140.2123084
---,---,---,---,---,---,---
,2018-02-10 07:54:22,16.509 sec,46.0,29178.9714116,16969.9368126,851412372.6371613
,2018-02-10 07:54:22,16.555 sec,47.0,29052.7063699,16885.4155057,844059747.4181669
,2018-02-10 07:54:22,16.601 sec,48.0,28938.7698949,16825.7123994,837452403.0285465
,2018-02-10 07:54:22,16.648 sec,49.0,28922.2740730,16843.6114655,836497937.5527190



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
OverallQual,86696372731904.0000000,1.0,0.2459078
ExterQual,41015142514688.0000000,0.4730895,0.1163364
GrLivArea,38298697334784.0000000,0.4417566,0.1086314
GarageCars,38004643069952.0000000,0.4383649,0.1077973
Neighborhood,34224002301952.0000000,0.3947570,0.0970738
---,---,---,---
MiscVal,17048897536.0000000,0.0001967,0.0000484
MiscFeature,13933739008.0000000,0.0001607,0.0000395
Condition2,12257705984.0000000,0.0001414,0.0000348
Street,1516759296.0000000,0.0000175,0.0000043



See the whole table with table.as_data_frame()


In [7]:
# Make predictions
yhat_test = drf_baseline.predict(h_test)
yhat_test = yhat_test.as_data_frame()

drf prediction progress: |████████████████████████████████████████████████| 100%


/Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'MSZoning' has levels not trained on: [NA]
  warnings.warn(w)
/Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'Utilities' has levels not trained on: [NA]
  warnings.warn(w)
/Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'Exterior1st' has levels not trained on: [NA]
  warnings.warn(w)
/Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'Exterior2nd' has levels not trained on: [NA]
  warnings.warn(w)
/Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'KitchenQual' has levels not trained on: [NA]
  warnings.warn(w)
/Users/jofaichow/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'F

In [8]:
# Create Kaggle Submission
d_sub = pd.read_csv("./data/sample_submission.csv")
d_sub = pd.concat([d_sub['Id'].reset_index(drop=True), yhat_test], axis=1)
d_sub.columns = ['Id', 'SalePrice']
d_sub.head(5)

,Id,SalePrice
0,1461,124430.333281
1,1462,155592.000000
2,1463,177635.099062
3,1464,185549.633437
4,1465,187703.900000


In [9]:
# Write to CSV
d_sub.to_csv("./kaggle_submission/baseline_submission.csv", index = False)